In [1]:
import import_ipynb
import os
import pandas as pd
import gensim
from collections import Counter
from pprint import pprint

from WNDomains import WordNetDomains 
wnd = WordNetDomains(wordnet_home='.')

importing Jupyter notebook from WNDomains.ipynb


In [2]:
import nltk
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english') + ['return']

In [ ]:
import spacy
nlp = spacy.load('en')

path= '../trame'

#domains_trama = set()  


list1=[1990, 2000, 2010]
dict_all_domains={}

for dirpath, dirnames, files in os.walk(path):
    dict_domains = {}
    
    for file in files:
        if file[0] == '.':
            continue

        domains_trama_all = set()
        counter = Counter()

        with open(dirpath+ "/" + file, 'r', encoding='Latin-1') as f:
            text = f.read()
            doc = nlp(text)
            #words = set([token.text if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in ['NOUN']])
            words = set([token.text for token in doc if (len(token.text) > 2 and token.pos_ in ['NOUN', 'PROPN'])])
            #words = set(gensim.utils.simple_preprocess(text, deacc=True))
            words = words.difference(en_stopwords) 

            file=file.split('.')[0]
            decade=dirpath.split('/')[-1]

            for word in words:
                domain = wnd.get_domains(word, 'n', first_sense_only=True)
                if 'factotum' in domain:
                    continue
                    
                for tmp_d in domain:
                    counter[tmp_d] += 1
                if len(domain) > 0:
                    print(file, word, '->', domain)
                domains_trama_all |= domain
                
            print(file)
            pprint(counter)

            for domain in domains_trama_all:
                if domain in dict_domains:
                    dict_domains[domain].append(file)
                else:
                    dict_domains[domain] = [file]

    for d in dict_domains:
        if decade in list1:
            if len(dict_domains[d]) >= (len(files)/2):
                if d in dict_all_domains:
                    dict_all_domains[d].append(decade)
                else:
                    dict_all_domains[d] = [decade]
        else:
            if len(dict_domains[d]) >= (len(files)*(0.75)):
                if d in dict_all_domains:
                    dict_all_domains[d].append(decade)
                else:
                    dict_all_domains[d] = [decade]
                                    

In [16]:
full_df = pd.DataFrame(columns=['1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010'])

for dom, decs in dict_all_domains.items():
    full_df.loc[dom] = ['Y' if (str(dec) in decs) else 'N' for dec in range(1930, 2010 +1, 10)]
    
full_df.sort_index(inplace=True)

In [17]:
full_df

,1930,1940,1950,1960,1970,1980,1990,2000,2010
administration,N,N,N,N,Y,N,N,N,N
anatomy,N,N,N,N,N,Y,N,N,N
animals,Y,N,Y,Y,Y,Y,Y,Y,Y
anthropology,Y,N,N,N,N,N,N,N,N
biology,Y,N,Y,Y,Y,Y,Y,Y,Y
buildings,Y,N,Y,Y,Y,Y,Y,Y,Y
chemistry,Y,N,N,N,Y,Y,N,Y,Y
dance,Y,N,N,N,N,N,N,N,N
economy,Y,N,N,Y,N,Y,N,N,N
electricity,Y,N,N,N,N,N,N,N,N


In [6]:
# DA NON FARE ANDARE
#tentativo inutile e cmq non riuscito cmq di ordinare per anno
df=df.fillna(0)
columns=list(df.columns)

for column in columns:
    year = 1930
    for j in range (0, 9):
        if df.iloc[j, df.columns.get_loc(column)] == 0:
            year+=1
            continue
        df.iloc[j, df.columns.get_loc(column)] = year
        year += 10

NameError: name 'df' is not defined

In [ ]:
df_null = df.isnull().sum(axis = 0)
df_null=df_null.to_frame().reset_index()

In [ ]:
df_null.rename(columns={'index': 'domain', 0:'counts'}, inplace=True)
df_null

In [ ]:
# se uno vuol fare dei check sui valori nulli (ossia )
for g in range (0,95):
    if df_null['counts'][g] > 5:
        print(df_null['domain'][g])

In [ ]:
df.to_csv(r"/Users/valeriaguttilla/Desktop/MASTER/PROGETTONE/ANALISI_TESTUALE/ANALISI TRAME/TRAME/ANIMATED/csv_trame.csv")